In [ ]:
import os
import glob
import json
import matplotlib.pyplot as plt

import plotly
plotly.offline.init_notebook_mode(connected=False)
import plotly.graph_objs as go
from plotly import tools

In [ ]:
dirs = glob.glob('output/*')
dirs.sort()

fig = tools.make_subplots(1, 2, shared_yaxes=True, print_grid=False, subplot_titles=['training', 'validation'])
colors = iter(['#4363d8', '#f58231', '#fabebe', '#e6beff', '#800000', '#000075', '#a9a9a9', '#ffe119', '#000000'])

fig2, axes = plt.subplots(2, 2, figsize=(12, 5))

for dir in dirs:

    csvpath = os.path.join(dir, 'history.csv')
    if not os.path.exists(csvpath):
        continue
        
    with open(os.path.join(dir, 'args.json'), 'r') as fp:
        args = json.load(fp)
    noise = args['noise']
    if args['optimizer'] != 'adam':
        continue

    color = next(colors)
    name = dir.replace('output/', '')
    df = pd.read_csv(csvpath, index_col=0)
    df.index += 1
    param = dict(line=dict(color=color), name=f'{name} ({noise})', legendgroup=name, mode='lines') 
    fig.add_trace(go.Scatter(x=df.index, y=df.loss, **param), 1, 1)
    fig.add_trace(go.Scatter(x=df.index, y=df.val_loss, **param, showlegend=False), 1, 2)
    
    df.loss.plot(ax=axes[0, 0], label=f'[-{noise}, {noise}]', marker='.')
    df.loss.plot(ax=axes[1, 0], marker='.')
    df.val_loss.plot(ax=axes[0, 1], marker='.')
    df.val_loss.plot(ax=axes[1, 1], marker='.')
    
fig['layout'].update(height=300)
fig['layout']['yaxis'].update(title='loss (MSE)', hoverformat='.5f', range=[0, 0.0015])
plotly.offline.iplot(fig)

axes[0, 0].legend(loc='best')
axes[0, 0].set_ylim(0, 0.4)
axes[0, 1].set_ylim(0, 0.4)
axes[1, 0].set_ylim(0, 0.0008)
axes[1, 1].set_ylim(0, 0.0008)
for ax in axes.flatten():
    ax.set_ylabel('loss (MSE)')
fig2.tight_layout()
fig2.savefig('output/loss.eps')